In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
q = data_frame = pd.read_parquet("relevance_challenge_train-001.parquet")

q.head()

,text,coordinates,label,raw_prediction,confidence,distance
20964,@joel271161 @poulin2012 vous êtes tellement fa...,"[0.695310096918557, 43.50579007522291]",721,"[-1.8304776, -1.6434574, -1.0601386, -1.431044...",0.032404,432.374286
448207,@sighutsu @blue01530961 那以前比較可以天天撿到幸運草～,"[121.39972544895538, 24.845272660654956]",953,"[-0.18133685, 0.07034852, 0.40745744, -0.15412...",0.254868,31.319643
497842,precisando viajar,"[-38.9130606409334, -13.071335826421613]",133,"[3.215739, -1.3704515, -0.6494701, -1.4568123,...",0.016078,1194.078186
453073,@memorylanerv @emc_maine @oh_grow_on @gardenin...,"[-82.8151652958084, 39.99343560927911]",1511,"[0.23749076, -0.8375926, 0.8497064, 1.0164264,...",0.387185,1114.610179
366398,@inemexico como que no son muy puntuales para ...,"[-103.47883507502031, 21.038488711387007]",129,"[-3.4438925, -2.687308, -0.7648393, -1.0434728...",0.057140,486.739171


In [3]:
df = q.sample(frac=0.1)

In [20]:
np.stack(df['raw_prediction']).shape

(49815, 3000)

In [23]:
q.iloc[1]['raw_prediction'].max()

8.807965

In [5]:
X = df['raw_prediction'].values
y = df['distance'].values

In [6]:
desired_length = 3000  # replace with the length that suits your data

# Function to truncate or pad the arrays
def transform_array(arr, length):
    if len(arr) > length:
        return arr[:length]  # Truncate
    else:
        return np.pad(arr, (0, length - len(arr)), 'constant')  # Pad

# Apply this transformation to each array
X = np.array([transform_array(arr, desired_length) for arr in X])


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [8]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(
    n_estimators=50,          # Reduced number of trees
    max_features='sqrt',      # Use a subset of features for each split
    min_samples_split=4,      # Increase the step size of tree building
    min_samples_leaf=2,       # Increase the minimum number of samples required at a leaf node
    max_depth=10,             # Limit the depth of trees
    criterion='absolute_error',          # Use a faster criterion (mean absolute error)
    n_jobs=-1,                # Use all available cores for parallel processing
    random_state=42,
    verbose=2
)


In [9]:
model.fit(X_train, y_train)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


building tree 1 of 50building tree 2 of 50

building tree 3 of 50
building tree 4 of 50
building tree 5 of 50
building tree 6 of 50
building tree 7 of 50
building tree 8 of 50


KeyboardInterrupt: 

In [10]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import pandas as pd


In [11]:
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)


LinearRegression()

In [12]:
y_pred = linear_model.predict(X_test)

# Calculate metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')


Mean Squared Error: 10893523.68460812
R^2 Score: 0.3762599053631164


In [13]:
import pandas as pd

# Assuming y_test are your actual values and y_pred are your model's predictions
results_df = pd.DataFrame({
    'actual': y_test,
    'predicted': y_pred
})

# Sort the DataFrame by the predicted values
sorted_results_df = results_df.sort_values(by='predicted')

# Select the lowest 10%
num_to_select = int(len(sorted_results_df) * 0.1)  # 10% of your dataset
selected = sorted_results_df.head(num_to_select)

# Calculate the average of the actual values
average_actual = selected['actual'].mean()

print(f'Average of actual values for lowest 10% predicted: {average_actual}')


Average of actual values for lowest 10% predicted: 598.7477213437182


In [14]:
from joblib import dump

# Assuming your model variable is named linear_model
dump(linear_model, 'linear_model.joblib')

['linear_model.joblib']